### Build A simple LLM Application with LCEL

1. Using Language Model
2. Using PromptTemplates and OutputParsers
3. Using Langchain Expression Language(LCEL) to chain components togeather
4. Debugging and Tracing your Application using Langsmith
5. Deploying your application with langserve


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

model = ChatGroq(
    model="gemma2-9b-it",
    groq_api_key=groq_api_key
)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F841DF6620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F841DF5660>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from english to french, make sure please don't give any other extra informations."),
    HumanMessage(content="Hello How are you?")
]

response1 = model.invoke(messages)
response1

AIMessage(content='Bonjour, Comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 34, 'total_tokens': 45, 'completion_time': 0.02, 'prompt_time': 0.000308109, 'queue_time': 0.014099831, 'total_time': 0.020308109}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-851a0e36-4830-417a-934f-0c660e6c6cbe-0', usage_metadata={'input_tokens': 34, 'output_tokens': 11, 'total_tokens': 45})

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser.invoke(response1)

'Bonjour, Comment allez-vous ? \n'

### Using LCEL We can chain the components

In [8]:
chain = model| parser

response_with_chain = chain.invoke(messages)
response_with_chain


'Bonjour, comment allez-vous ? \n'

### Prompt Templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate

generaic_template = "Translate the following into {language}:"
text = "Hello"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generaic_template),
        ("user", "{text}")
    ]
)

chain = prompt | model | parser

chat_template_response = chain.invoke({"language": "French", "text": text})
chat_template_response

'Bonjour  (formal)\n\nSalut (informal)\n'

In [12]:
chat_template_response2 = chain.invoke({"language": "Hindi", "text": text})
chat_template_response2

'नमस्ते (Namaste) \n'